In [70]:
from io import BufferedReader
import struct

# Utility reader subclass
class WBR(BufferedReader):
  """
  Collection of macros for smoothly translating RTB's MaxScript
  
  Most of the time long/short ints are unsigned
  """
  
  def readLong(self, signed = False):
    """Read long int"""
    return struct.unpack('l' if signed else 'L', self.read(4))[0]

  def readLongSigned(self):
    """Shorthand for readLong(signed=True)"""
    return self.readLong(True)

  def readByte(self, signed = False):
    """Read short int"""
    return struct.unpack('h' if signed else 'H', self.read(2))[0]

  def readString(self, n : int):
    return ''.join([x.decode() for x in struct.unpack('c'*n, self.read(n))])
  
  def readFloat(self):
    """Read Float"""
    return struct.unpack('f', self.read(4))[0]
  
  def seek_rel(self, offset):
    """Seek relative to current position"""
    return super().seek(offset, 1)
  
  def seek_abs(self, offset):
    """Seek relative to 0"""
    return super().seek(offset, 0)

In [71]:
f = open(r"C:\Users\woas\Downloads\d3d\input\obj_carTaxiA.d3dmesh", 'rb')
f = WBR(f)


VerboseLevel = 1
header = f.readLong()
HeaderMagic = header.to_bytes(4).decode('ascii')
print(f"HeaderMagic = {HeaderMagic}")
FileSize = f.readLong()
print(f"FileSize = {FileSize}")
f.seek_rel(0x08)
ParamCount = f.readLong()
print(f"ParamCount = {ParamCount}")
for x in range(ParamCount):
  if VerboseLevel > 2:
    print(f"Unknown param {f.read(0x0C).decode('ansi')}")
  else:
    f.seek_rel(0x0C)
D3DNameHeaderLength = f.readLong()
D3DNameLength = f.readLong()
if D3DNameLength > D3DNameHeaderLength:
  f.seek_rel(-0x04)
  D3DNameLength = D3DNameHeaderLength

print(f"D3DNameHeaderLength {D3DNameHeaderLength}, D3DNameLength {D3DNameLength}")
D3DName = f.readString(D3DNameLength)
VerNum = f.readByte()
print(f"Importing {D3DName} (Version {VerNum})...")
MatInfoStart = f.tell() + 0x13 #skipping Section 1 (Model Info)

MSV6
MSV6...
FileSize = 295465
ParamCount = 29
D3DNameHeaderLength 28, D3DNameLength 20
Importing obj_carTaxiA.d3dmesh (Version 55)...


In [72]:
#TODO Find & Parse *.skl files

In [73]:
# Section 2 (Material Info)
f.seek_abs(MatInfoStart)
print(f"Section 2 (Material Info) start @{f.tell()}")
MatCount = f.readLong()
print(f"Material Count = {MatCount}")

#Parsing Material Info
for x in range(MatCount):
  print(f"Material #{x} @{f.tell()}")
  MatHash2 = f.readLong()
  MatHash1 = f.readLong()
  UnkHash2 = f.readLong()
  UnkHash1 = f.readLong()
  MatHeaderSize = f.tell() + f.readLong()

  MatUnk1 = f.readLong()
  MatUnk2 = f.readLong()
  MatHeaderSizeB = f.readLong()

  MatUnk3Count = f.readLong()
  for m in range(MatUnk3Count):
    MatUnk3Hash2 = f.readLong()
    MatUnk3Hash1 = f.readLong()

  MatParamCount = f.readLong()
  TexDifName = "undefined"
  print(f"Material Parameter Count = {MatParamCount}")
  for x in range(MatCount):
    #TODO parse Material Params
    pass

  f.seek(MatHeaderSize)
print(f"Section 2 (Material Info) end @{f.tell()}")

Section 2 (Material Info) start @417
Material Count = 9
Material #0 @421
Material Parameter Count = 6
Material #1 @4336
Material Parameter Count = 7
Material #2 @8371
Material Parameter Count = 6
Material #3 @12293
Material Parameter Count = 6
Material #4 @16144
Material Parameter Count = 7
Material #5 @20304
Material Parameter Count = 7
Material #6 @24464
Material Parameter Count = 7
Material #7 @28624
Material Parameter Count = 6
Material #8 @32546
Material Parameter Count = 6
Section 2 (Material Info) end @36468


In [74]:
#FaceDataStart + Section 3 (LOD)
unk = f.readLong()
pad = f.readLong()
FaceDataStart = f.tell() + f.readLong() #WOAS: I'd just like to point out how random it is for this pointer to be here of all places, can't imagine how RTB figured this out

Sect3End = f.tell() + f.readLong()
Sect3Count = f.readLong()
print(f"Section 3 (LOD info) start @ {f.tell()-8}, Count = {Sect3Count}")

#TODO Parse LOD Info

Section 3 (LOD info) start @ 36480, Count = 204800


In [75]:
f.seek(Sect3End)
Sect4End = f.tell() + f.readLong()
print(type(f))

<class '__main__.WBR'>


In [76]:
f.close()